<a href="https://colab.research.google.com/github/wowothk/jupyter/blob/master/%5BLembar_Kerja%5D_DSC_UI_Summer_School_2020_ML_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dasar Pembelajaran Mesin / Machine Learning Basics

**Bramantyo Adrian & Dimitrij Ray**

**AI Engineer, GDP Labs**

**Senin, 19 Oktober 2020**

# Problem
Dataset yang akan digunakan adalah dataset `Adult` dari [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult). Dataset ini berisi sampel hasil sensus di Amerika Serikat yang diadakan tahun 1994. 

Kita akan menggunakan dataset ini untuk melatih yang dapat memprediksi apakah seseorang memiliki penghasilan lebih dari USD 50000 per tahun.

Atribut di dalam data:
- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous. (final weight)
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.


**Sebelum Anda mulai:**

Apabila Anda menggunakan Google Colaboratory, silakan jalankan sel berikut untuk membaharui beberapa *library* yang akan kita gunakan selama sesi ini.

Jika Anda menggunakan Jupyter Notebook atau JupyterLab, silakan hapus sel ini.

In [ ]:
!pip install --upgrade numpy scipy pandas scikit-learn seaborn

## Mengimpor *library*

Untuk sesi praktik ini, kita akan menggunakan beberapa *library* yang umum digunakan untuk keperluan pembelajaran mesin, yaitu:
1. `matplotlib` dan `seaborn` untuk visualisasi data,
1. `numpy` dan `pandas` untuk penampungan dan pemrosesan data, serta
1. `sklearn` untuk pemodelan.

Jalankan sel berikut untuk mengimpor *library*-*library* tersebut. Kita akan terlebih dahulu mengimpor modul-modul yang dibutuhkan untuk pemrosesan data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

## Memuat dataset

Lengkapi dan jalankan sel berikut untuk memuat dataset yang berformat `csv` ini ke dalam sebuah `DataFrame`. Perhatikan bahwa:

1. Ada 2 macam berkas yang kita muat: `adult.data` yang berisi data latih dan `adult.test` yang berisi data uji.
1. Masing-masing berkas `csv` **tidak** memiliki *header*, atau penunjuk nama-nama kolom di baris pertama. Oleh karena itu, melalui argumen `names` pada *method* `read_csv`, kita perlu memasukkan nama-nama kolomnya. 
  - Dalam situasi nyata, Anda harus mencocokkan nama kolom dengan deskripsi yang diberikan oleh pemilik data, tapi kali ini hal tersebut telah dilakukan untuk Anda.
  - Dalam situasi nyata Anda juga mungkin menemukan berkas `csv` yang tidak terformat dnegan baik. Oleh karena itu, saat Anda menerima berkas, periksa terlebih dahulu menggunakan *text processor* favorit Anda (mis. Notepad++, Sublime Text).

In [ ]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']

data_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', names=col_names, delimiter=', ')

## <--- LENGKAPI --->
## Lengkapi sumber data untuk method read_csv, serta argumen names dan delimiter.
## Kita akan membaca data uji dari https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

test_df = pd.read_csv(None, skiprows=1, names=None, delimiter=None)


Gunakan *method* `head(n)` untuk melihat $n$ baris pertama dari sebuah dataset.

In [ ]:
data_df.head(5)

Kita bisa menggunakan *method* `notnull()` untuk mengetahui apakah ada baris-baris yang kosong.

In [ ]:
train_rows = len(data_df)
train_without_missing = len(data_df.notnull())

## <--- LENGKAPI --->
## Hitunglah banyak baris di test_df, serta banyaknya baris yang tak-kosong.
test_rows = None
test_without_missing = None

print(f'Train rows: {train_rows}; missing values: {train_rows - train_without_missing}')
print(f'Test rows: {test_rows}; missing values: {test_rows - test_without_missing}')

## Memisahkan data latih dan data validasi

Dalam praktik pembelajaran mesin, apabila kita memiliki data latih yang cukup banyak, kita dapat menyisihkan sebagian data latih untuk digunakan sebagai data validasi.  Data validasi nanti akan kita gunakan untuk mengestimasi galat data uji.

Biasanya pemisahan data latih, data validasi, dan data uji dilakukan dengan acak.  Akan tetapi, ada beberapa situasi yang memungkinkan dilakukannya cara lain untuk memisahkan dataset yang kita miliki.
1. Apabila masalah yang akan kita selesaikan adalah masalah klasifikasi dan terjadi ketimpangan sampel data yang berasal dari kelas tertentu (situasi ini disebut *class imbalance*), kita bisa melakukan pemisahan menggunakan prinsip *stratified sampling* supaya proporsi masing-masing kelas terjaga untuk tiap-tiap partisi data.
1. Apabila dataset dan masalah yang dihadapi berupa deret waktu (*time series*), kita dapat melakukan pemisahan berdasarkan satuan waktu tertentu (misalnya kita menggunakan data dari bulan 1 - 4 untuk melatih model dan bulan ke-6 untuk mengujinya).

Kita sekarang menghadapi masalah klasifikasi. Oleh karenanya, baik apabila kita melihat terlebih dahulu apakah kita menghadapi situasi *class imbalance* atau tidak.  Untuk itu, kita dapat menggunakan *method* `groupby()` dan `count()` untuk mengelompokkan data kita berdasarkan kelas dan menghitung banyaknya data yang ada di kelas tersebut.

In [ ]:
data_df.groupby('class').count()

Dapat dilihat bahwa kita mayoritas orang dalam dataset berada di kelas pertama, yakni `<=50K`. Oleh karena itu, kita menggunakan teknik *stratified sampling* untuk membuat data validasi.

In [ ]:
train_df, val_df = train_test_split(data_df, test_size=0.3, stratify=data_df['class'], random_state=42)

In [ ]:
## <--- LENGKAPI ---> 
## Gunakan .groupby() dan .class() untuk menghitung banyaknya baris di masing-
## masing kelas di DataFrame train_df.  Bandingkan perbandingan antara kelas
## '<=50K' dan '>50K' di data_df dan di train_df.

In [ ]:
## <--- LENGKAPI ---> 
## Lakukan hal yang sama untuk val_df.

Mari kita lihat beberapa baris pertama data latih dan data validasi.

In [ ]:
## <--- LENGKAPI --->
## Cobalah melihat 5 baris pertama dari train_df.

None

In [ ]:
## <--- LENGKAPI --->
## Cobalah melihat 5 baris pertama dari val_df.

None

Sekilas, ada dua hal yang perlu diperhatikan:
- Terdapat beberapa nilai `?`, yang sepertinya menyatakan nilai yang hilang (*missing values*)
- Kelas target kita, variabel `class`, berbeda format di data uji.

Masalah yang kedua dapat kita selesaikan dengan mudah saat melakukan *preprocessing*. Masalah yang pertama perlu kita teliti lebih lanjut. Mari kita lihat berapa baris untuk tiap kolom di data latih yang memiliki nilai `?` dan simpan kolom-kolom yang memiliki nilai `?`.

In [ ]:
col_with_missing = list()
for col in train_df.columns:
  try:
    missings = len(train_df[train_df[col] == '?'])
    print(f'Column {col} has {missings} missing value(s)')
    if missings != 0:
      col_with_missing.append(col)
  except ValueError:
    print(f'Column {col} seems to have no missing values, since it is not of type string.')

Terlihat bahwa beberapa kolom kategorikal memiliki nilai `?`.  Untuk variabel kategorikal, nilai kosong dapat dijadikan kategori baru atau diimputasi.  Untuk sesi ini, nanti kita akan memperlakukan nilai `?` sebagai kategori baru.

## Bagian 1: Analisis data eksploratif / *Exploratory data analysis* (EDA)

Hal pertama yang harus dilakukan sebelum kita mulai melakukan transformasi dan pemodelan adalah menganalisis data yang ada.

Pada bagian ini kita hanya akan melihat sedikit saja eksplorasi yang bisa dilakukan terhadap dataset ini. Anda dipersilakan untuk mengeksplorasi sendiri setelah sesi ini.

### Analisis atribut

Pertama, kita dapat melihat kolom apa saja yang tersedia beserta tipenya (apakah akan kita perlakukan sebagai variabel kontinu atau kategorikal). Selain itu, apabila tersedia, kita juga dapat mencocokkan nilai-nilai setiap kolom dengan spesifikasi dataset tersebut.

In [ ]:
train_df.columns

In [ ]:
train_df.dtypes

`DataFrame` `pandas` memiliki *method*
 `describe()` yang dapat membantu menghitung statistik deskriptif dari suatu dataset. Untuk kolom-kolom yang diperlakukan sebagai variabel kontinu, `describe()` mengeluarkan mean, standar deviasi, nilai minimum dan maksimum, serta nilai kuartil.

Untuk membuang `continuous_columns` dan `ordinal_columns` dari `train_df.columns`, kita dapat memanfaatkan tipe data `set` dari Python. Operasi `set(a) - set(b)` akan mengeluarkan elemen-elemen yang ada di `a` tetapi tidak di `b`.

Untuk memilih kolom-kolom tertentu saja dari suatu `DataFrame`, kita dapat menggunakan `df[['a', 'b', 'c', 'd']]` untuk memilih kolom `a`, `b`, `c`, `d` di `DataFrame` `df`.

In [ ]:
continuous_columns = ['capital-gain', 'capital-loss', 'hours-per-week', 'age', 'fnlwgt']
ordinal_columns = ['education', 'education-num']
label_col = 'class'
categorical_columns = sorted(list(set(train_df.columns) - set(continuous_columns + ordinal_columns + [label_col])))

continuous_df = train_df[continuous_columns]
continuous_df.describe()

Untuk kolom-kolom yang bertipe `object`, `describe()` akan menghitung banyaknya nilai yang unik dan nilai yang sering muncul beserta frekuensinya.

In [ ]:
## <--- LENGKAPI --->
## Hitunglah statistik deskriptif label (variabel class) dan kolom yang termasuk 
## dalam list categorical_columns di data latih.

categorical_df = None
None

Kita dapat menggunakan kelas `Categorical` dari `pandas` untuk mengubah suatu kolom yang tadinya bertipe `int` menjadi tipe `category`.  Metode `describe()` akan mengeluarkan keluaran yang sama seperti ketika kita menggunakannya untuk kolom yang bertipe `object`.

In [ ]:
ordinal_data = train_df[ordinal_columns]
ordinal_data.loc[:, ['education-num']] = pd.Categorical(ordinal_data['education-num'])
print(ordinal_data.dtypes)

ordinal_data[ordinal_columns].describe()

### Analisis univariat

Dari analisis sederhana tadi, ada setidaknya tiga hal yang menarik perhatian:
1. Standar deviasi `capital-gain` tinggi sekali, mencapai 7552, padahal kuartil-kuartilnya 0.
1. Variabel `education-num` dan `education` memiliki nilai paling sering dengan frekuensi yang sama.
1. Terdapat banyak variabel kontinu yang berbeda satuan/skala.

Temuan pertama dapat kita lihat lebih jauh menggunakan analisis univariat. Temuan kedua dapat kita telusuri lebih jauh menggunakan analisis bivariat. Temuan ketiga langsung dapat kita tindak lanjuti saat melakukan rekayasa fitur dengan cara melakukan *scaling*.

Mari kita mulai dengan menginvestigasi `capital-gain`. Kita dapat menggunakan *method* `hist(column: list, figsize: tuple, layout: tuple, bins: int)` di suatu `DataFrame` untuk membuat histogram secara otomatis.

In [ ]:
train_df.hist(column="capital-gain", figsize=(10,30), layout=(5,1), bins=20)
plt.show()

Terlihat bahwa sepertinya sebagian besar nilai variabel `capital-gain` berada di bawah 40000, akan tetapi ada sebagian kecil yang memiliki nilai di atas 90000.  Sepertinya kita berhadapan dengan pencilan. Mari kita lihat beberapa sampel yang memiliki nilai lebih besar dari atau sama dengan 90000.  Kita dapat menggunakan *method* `sample(n)` untuk mengambil *n* buah sampel tanpa pengembalian.

In [ ]:
train_df_high_capital = train_df[train_df['capital-gain'] >= 90000]

train_df_high_capital.sample(10, random_state=21).head(10)

Kita juga dapat menggunakan *method* `len` untuk menghitung banyaknya baris di suatu `DataFrame`.

In [ ]:
print(len(train_df_high_capital))

Dari sampel, terlihat bahwa orang-orang dengan `capital-gain` 99999 ini adalah orang-orang dengan posisi karier yang cukup tinggi, misalnya guru besar atau bagian eksekutif-manajerial.  Nilai 99999 sepertinya adalah nilai *cap* yang ditetapkan oleh badan sensus ketika `capital-gain` seseorang bernilai 100000 atau lebih.

Lebih dari itu, orang-orang ini sepertinya hampir pasti memiliki penghasilan di atas USD 50000. Oleh karena itu, sebaiknya pencilan-pencilan ini tidak dibuang, akan tetapi kita kontrol dengan cara melakukan *binning*.

## Analisis bivariat

Berikutnya, kita dapat menginvestigasi `education` dan `education-num`. Untuk mencari tahu apakah benar `education-num` dan `education` memiliki korespondensi satu-satu, kita dapat menggunakan tabel pivot dan fitur *heatmap* dari `seaborn`.

In [ ]:
train_df_dummy = train_df.copy()
train_df_dummy['dummy'] = 1
edu_pivot = pd.pivot_table(train_df_dummy, 
                           values='dummy', 
                           index='education-num', 
                           columns='education', 
                           aggfunc='count')

edu_pivot

Terlihat bahwa banyak sekali entri yang kosong, ditandai dengan `NaN` (*not a number*). Kita akan biarkan entri-entri ini kosong, sebab `seaborn` secara otomatis akan mengosongkan bagian tersebut juga saat membuat *heatmap*.

In [ ]:
plt.figure(figsize=(8, 7))
sns.heatmap(edu_pivot)

Terlihat dari *heatmap* bahwa memang setiap entri di `education-num` dan `education` saling bersesuaian. Oleh karena itu, nantinya ketika melakukan rekayasa fitur kita cukup menggunakan satu kolom saja.

Ada pula beberapa jenis analisis bivariat lain yang dapat dilakukan. Sebagai contoh, kita ingin tahu apakah variabel `fnlwgt` memiliki pengaruh terhadap `class`.

Hal pertama yang bisa lakukan adalah menggambar *box plot*, yang akan menunjukkan kuartil-kuartil `fnlwgt` berdasarkan `class`, serta pencilan yang dihitung menggunakan jarak antarkuartil (*interquartile range*).

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.boxplot(x='class', y='fnlwgt', data=train_df)
plt.show()

Dari *box plot*, terlihat bahwa sepertinya kuartil-kuartil `fnlwgt` mirip sekali di kedua kelas. Keduanya pun sepertinya merupakan distribusi yang menceng ke kanan (*right-skewed*). 

Oleh karenanya, kita mungkin dapat mengasumsikan bahwa variabel `fnlwgt` mungkin tidak dapat menjadi pembeda yang baik antara kedua kelas. Untuk mengecek apakah kita *mungkin* ingin berubah pikiran, kita dapat melakukan uji hipotesis.

In [ ]:
train_df[train_df['class'] == '>50K']['fnlwgt'].hist(bins=20)

In [ ]:
## <--- LENGKAPI --->
## Periksalah histogram fnlwgt untuk kelas '<=50K'.

None

Kita akan menggunakan Welch's $t$-test, yakni variasi uji-$t$ yang memiliki asumsi:
- kedua populasi berdistribusi normal,
- kedua populasi mungkin memiliki variansi yang tidak sama.

Karena distribusi yang kita miliki menceng ke kanan, kita dapat melakukan transformasi logaritma terhadap nilai `fnlwgt` sehingga kita mendapatkan distribusi yang lebih simetris. Kita asumsikan distribusi `log(fnlwgt)` ini adalah normal. Lengkapi dan jalankan sel-sel berikut untuk melihat histogram `log(fnlwgt)` untuk kedua kelas.

In [ ]:
np.log(train_df[train_df['class'] == '>50K']['fnlwgt']).hist(bins=20)

In [ ]:
## <--- LENGKAPI --->
## Buatlah histogram hasil log(fnlwgt) untuk kelas '<=50K' yang memiliki 20 bin.

None

Kita akan menggunakan modul `stats` *library* `scipy`.   

Hipotesis-hipotesis kita adalah sebagai berikut.

$H_0$: Mean dari `fnlwgt` di kedua kelas sama.

$H_1$: Mean dari `fnlwgt` di kedua kelas tidak sama.

Kita akan menggunakan taraf signifikansi $\alpha = 0.05$. 

Jalankan sel berikut untuk mengimpor *method* `ttest_ind` `scipy.stats` dan menjalankan uji-$t$.

In [ ]:
from scipy.stats import ttest_ind

positive = np.log(train_df[train_df['class'] == '>50K']['fnlwgt'])
negative = np.log(train_df[train_df['class'] == '<=50K']['fnlwgt'])

test_stat, pval = ttest_ind(positive, negative, equal_var = False)

print("Test statistic (T) :", test_stat)
print("P-value: ", pval)

Kita melihat *p-value* yang lebih tinggi daripada taraf signifikansi yang telah ditetapkan di awal. Oleh karena itu, kita memutuskan bahwa hipotesis awal kita tidak ditolak.  Ini berarti: data yang kita miliki tidak cukup meyakinkan kita untuk menolak asumsi awal kita bahwa mean dari `fnlwgt` di kedua kelas adalah sama.

Oleh karena itu, karena kita menginginkan model yang lebih sederhana, kita tidak akan menggunakan variabel `fnlwgt`.

# Bagian 2: *Preprocessing* dan rekayasa fitur (*feature engineering*)

Berdasarkan hasil analisis data, kita dapat melakukan tahap selanjutnya, yaitu ***preprocessing* & rekayasa fitur**. Kita akan:

1. Membuang kolom yang tidak akan digunakan,
1. Menyederhanakan nilai-nilai atribut kategorikal,
1. Melakukan *one-hot encoding* untuk atribut kategorikal,
1. Normalisasi, dan
1. *Binning* atribut numerik

### Membuang kolom yang tidak digunakan

Dari hasil analisis data, kita memutuskan:
- Akan menggunakan salah satu saja dari kolom `education` dan `education-num`; dalam kasus ini kita akan mengambil kolom `education-num` saja karena memiliki informasi urutan.
- Tidak akan menggunakan `fnlwgt`.

Oleh karena itu, kita akan membuang kolom `fnlwgt` dan `education`. Kita dapat menggunakan *method* `drop(columns: list)` untuk melakukan ini.

In [ ]:
## <--- LENGKAPI --->
## Berdasarkan deskripsi di atas, lengkapi variabel dropped_cols dan buang 
## dropped_cols dari train_df.

dropped_cols = None
prep_train_df = train_df.drop(columns=None)
prep_train_df.head()

### Menyederhanakan nilai-nilai atribut kategorikal

Kebanyakan modul pembuatan model hanya akan menerima masukan label dalam bentuk numerik, umumnya 0 dan 1. Oleh karena itu, kita perlu mengubah label di variabel `class` menjadi nilai 0 dan 1.

Selain itu, perhatikan bahwa kita memiliki beberapa atribut kategorikal yang memiliki nilai cukup banyak, misalnya `native-country` dan `marital-status`. Karena nanti kita akan menggunakan metode *one-hot encoding*, variasi nilai yang banyak akan mengakibatkan pembuatan kolom baru yang juga banyak, sehingga menambah kompleksitas model.  Oleh karena itu, kita akan menyederhanakan kedua variabel tersebut dengan cara memetakan nilai yang lama ke nilai yang baru.

Kedua masalah ini dapat diselesaikan dengan *method* `map(mapper: dict)`.

In [ ]:
def map_categorical_values(df: DataFrame, col_map_pairs: [(str, dict)]):
    result_df = df.copy()
    for col, map_ in col_map_pairs:
      result_df[col] = result_df[col].map(map_)
      
    return result_df

In [ ]:
def map_label(df: DataFrame, label_col: str, positive_label: str, negative_label: str):
  result_df = df.copy()
  result_df[label_col] = result_df[label_col].map({positive_label: 1, negative_label: 0})

  return result_df

In [ ]:
col_map_pairs = [
    (
        'marital-status', {
            'Married-civ-spouse': 'Couple', 
            'Divorced': 'Single',
            'Never-married': 'Single',
            'Separated': 'Single', 
            'Widowed': 'Single',
            'Married-spouse-absent': 'Single',
            'Married-AF-spouse': 'Couple'
        }
    ),
    (
        'native-country', {
          'United-States': 'United-States', 
          'Cambodia': 'Southeast-Asia', 
          'England': 'Europe', 
          'Puerto-Rico': 'America', 
          'Canada': 'Canada', 
          'Germany': 'Europe',
          'Outlying-US(Guam-USVI-etc)': 'America', 
          'India': 'Asia', 
          'Japan': 'Asia', 
          'Greece': 'Europe', 
          'South': 'Asia', 
          'China': 'Asia', 
          'Cuba': 'America', 
          'Iran': 'Asia', 
          'Honduras': 'America', 
          'Philippines': 'Southeast-Asia', 
          'Italy': 'Europe', 
          'Poland': 'Europe', 
          'Jamaica': 'America', 
          'Vietnam': 'Southeast-Asia', 
          'Mexico': 'America', 
          'Portugal': 'Europe', 
          'Ireland': 'Europe', 
          'France': 'Europe', 
          'Dominican-Republic': 'America', 
          'Laos': 'Southeast-Asia', 
          'Ecuador': 'America', 
          'Taiwan': 'Asia', 
          'Haiti': 'America', 
          'Columbia': 'America', 
          'Hungary': 'Europe',
          'Guatemala': 'America', 
          'Nicaragua': 'America', 
          'Scotland': 'Europe', 
          'Thailand': 'Southeast-Asia', 
          'Yugoslavia': 'Europe', 
          'El-Salvador': 'America', 
          'Trinadad&Tobago': 'America', 
          'Peru': 'America', 
          'Hong': 'Asia', 
          'Holand-Netherlands': 'Europe',
          '?': '?'
      }
    )
]
prep_train_df_label_mapped = map_label(prep_train_df, 'class', '>50K', '<=50K')
prep_train_df_simplified = map_categorical_values(prep_train_df_label_mapped, col_map_pairs)

In [ ]:
for col, _ in col_map_pairs:
  print("Unique values of {}: {}".format(col, prep_train_df_simplified[col].unique()))

### *One-hot encoding* untuk atribut kategorikal

*One-hot encoding* adalah salah satu cara yang kerap dipakai untuk memanipulasi fitur-fitur kategorikal agar bisa dimasukkan ke dalam sebuah model yang umumnya berupa suatu persamaan. 

Misalkan variabel `buah` memiliki nilai `['apel', 'belimbing', 'cempedak', 'unknown']`. Masing-masing nilai di variabel tersebut dapat dinyatakan sebagai vektor $v \in \{0, 1\}^4$ sebagai berikut:
- `apel`: $(1, 0, 0, 0)$
- `belimbing`: $(0, 1, 0, 0)$
- `cempedak`: $(0, 0, 1, 0)$
- `unknown`:  $(0, 0, 0, 1)$

Alternatifnya, kita dapat menyatakan masing-masing nilai sebagai vektor di $\{0, 1\}^3$ dengan memerlakukan (misalnya) `unknown` sebagai kategori "lain-lain", sehingga bentuk *encoding*-nya adalah:

- `apel`: $(1, 0, 0)$
- `belimbing`: $(0, 1, 0)$
- `cempedak`: $(0, 0, 1)$
- `unknown`: $(0, 0, 0)$

Kita akan menggunakan konvensi yang *kedua*: jika variabel kategorikal kita memiliki $d$ variasi, maka kita akan mengkodekan variabel tersebut sebagai vektor di $\{0, 1\}^{d-1}$.

Kita akan menggunakan kelas `OneHotEncoder` dari `scikit-learn` untuk men-*encode* kolom-kolom di `categorical_columns`.

In [ ]:
print(categorical_columns)

Sebenarnya, karena kita sudah mengetahui kemungkinan isi dari masing-masing kolom berdasarkan deskripsi, kita bisa saja mendefinisikan kategori yang akan dikenali oleh `OneHotEncoder`. Perhatikan bahwa urutan memasukkan kategori untuk setiap kolom *harus* sesuai dengan urutan kolom yang akan di-*encode*.

Dalam praktiknya hal ini tidak selalu terjadi. Apabila kita tidak mengetahui kemungkinan isi dari masing-masing kolom, **tidak perlu** menggunakan argumen `categories`. Cukup panggil metode `fit`, dan `OneHotEncoder` akan secara otomatis belajar dari data latih yang Anda masukkan.

Untuk sesi ini, kita tidak akan mendefinisikan `categories`. 

Jalankan sel berikut untuk melihat hasil *one-hot encoding* kolom-kolom `categorical_columns`. Perhatikan bahwa ada dua *method* yang harus dipanggil: `fit()` untuk "melatih" `OneHotEncoder` mengenali kategori-kategori di dataset, dan `transform()` untuk melakukan *encoding*. Metode `fit`-`transform` ini akan sering kita lihat pada saat melakukan *preprocessing*.

In [ ]:
ohe_encoder = OneHotEncoder(drop='first', sparse=False)
ohe_encoder.fit(prep_train_df_simplified[categorical_columns])
ohe_encoder.transform(prep_train_df_simplified[categorical_columns])

### *Scaling* atribut kontinu

Salah satu penemuan kita pada saat analisis data adalah fakta bahwa banyak variabel kontinu yang memiliki skala berbeda. Agar model kita tidak terpengaruh terhadap skala, kita dapat melakukan *scaling*. Ada dua metode *scaling* sederhana yang kerap digunakan. Misalkan kita memiliki nilai $x$ di suatu variabel $X$:
1. *Min-max scaling*: mengurangi nilai $x$ dengan nilai minimum variabel $X$ di dataset, kemudian membaginya dengan jangkauan (*range*) variabel tersebut:
$$x^{(\text{scaled})} = \frac{x - X_\text{min}}{X_\text{max} - X_\text{min}}.$$
Dapat ditunjukkan bahwa metode *scaling* ini akan menghasilkan variabel yang berada pada rentang $[0, 1]$.
1. *Standardization* atau *standard scaling*: mengurangi nilai $x$ dengan (estimasi) mean variabel $X$, kemudian membaginya dengan (estimasi) standar deviasinya:
$$ x^{(\text{scaled})} = \frac{x - \mu_X}{\sigma_X}.
$$
Dapat ditunjukkan bahwa metode *scaling* ini akan menghasilkan variabel yang memiliki mean $0$ dan variansi $1$.

Metode *scaling* yang digunakan bergantung pada masalah dan dataset yang dihadapi. Untuk sesi ini, kita akan menggunakan *standard scaler*.

`scikit-learn` memiliki kelas `StandardScaler` yang dapat digunakan untuk keperluan ini. Untuk menggunakannya, pertama kita harus memanggil *method* `fit()` supaya objek *scaler* kita tahu mean dan standar deviasi kolom yang ingin kita transformasikan. Setelah itu, kita memanggil *method* `transform` untuk melakukan transformasinya.

In [ ]:
standardized_cols = ['age', 'hours-per-week']
scaler = StandardScaler()

## <--- LENGKAPI --->
## "Latih"lah scaler menggunakan kolom-kolom standardized_cols dari 
## prep_train_df_simplified, kemudian transformasikan kolom-kolom yang sama 
## menggunakan scaler yang telah di"latih".

None

None

Perhatikan bahwa `OneHotEncoder` dan `StandardScaler` hanya melakukan transformasi terhadap kolom-kolom yang dipilih saja, tetapi tidak mendukung transformasi satu DataFrame. Kita akan lihat nanti bagaimana cara mengatasi ini.

### *Binning* atribut kontinu

Hal lain yang dapat dilakukan terhadap variabel kontinu adalah melakukan *binning* atau pengelompokan. 

Ada bermacam-macam kriteria pengelompokan. Dua di antaranya yang paling sederhana adalah:
1. *Uniform intervals*, yakni pengelompokkan yang dilakukan sedemikian rupa sehingga panjang interval masing-masing kelompok sama.
2. *Quantile-based*, yakni pengelompokkan yang dilakukan sedemikian rupa sehingga masing-masing interval memiliki banyak sampel yang sama.

Kita juga bisa menentukan batas-batas interval setiap kelompok berdasarkan pengetahuan sebelumnya atau berdasarkan hasil analisis data.

`scikit-learn` memiliki kelas `KBinsDiscretizer` untuk melakukan *binning*. Hanya saja, kelas tersebut tidak mendukung pembuatan *bin* yang didefinisikan sendiri. Untuk keperluan sesi ini, Anda telah dibuatkan fungsi `bin_numerical_attributes` yang dapat menerima *bin* yang didefinisikan sendiri.  Kita akan menggunakan fungsi ini untuk mengelompokkan `capital-gain` dan `capital-loss`.

In [ ]:
def _map_value_to_bin(value: int, bin_thresholds: [float]):
  count = 0
  
  for threshold in bin_thresholds:
    if value < threshold:
      return count
    else:
      count += 1
    

def bin_numerical_attributes(df: DataFrame, col_bin_pairs: [(str, [float])] ):
    result_df = df.copy()
    for col, bin_thresholds in col_bin_pairs:
      result_df[col] = result_df[col].apply(lambda val: _map_value_to_bin(val, bin_thresholds))
    
    return result_df

In [ ]:
bin_capital_gain = [0.0, 1.0, 3000.0, 5000.0, 10000.0, 20000.0, float('inf')] 
bin_capital_loss = [0.0, 1.0, 1700, 1900.0, 2000, float('inf')]

col_bin_pairs = [('capital-gain', bin_capital_gain), ('capital-loss', bin_capital_loss)]

prep_train_df_binned = bin_numerical_attributes(prep_train_df_scaled, col_bin_pairs)

prep_train_df_binned.head()

### Menggunakan `ColumnTransformer`

Untuk menyatukan transformasi yang menggunakan kelas `scikit-learn` pada kolom berbeda-beda, kita dapat menggunakan kelas `ColumnTransformer`. Mari kita gunakan kelas ini untuk menyatukan `OneHotEncoder` dan `StandardScaler`.  Sebagai contoh, kita tetap akan menstandardisasi kolom-kolom di `standardized_cols`, tetapi kita hanya akan meng-`encode` kolom `sex`.  

Jalankan sel berikut, dan bandingkan hasilnya dengan baris ke-2 *one-hot encoding*, standardisasi, dan *binning* yang telah Anda lakukan sebelumnya.

In [ ]:
from sklearn.compose import ColumnTransformer

ct_example = ColumnTransformer(
    [
     ('standar-scaler-1', StandardScaler(), standardized_cols),
     ('ohe-1', OneHotEncoder(drop=None, sparse=False), ['sex'])
    ],
    remainder='passthrough',
    verbose=True
     )

ct_example.fit(bin_df)

ct_example.transform(bin_df)[1]

## Bagian 3: Melatih model

Setelah kita selesai melakukan pemrosesan, saatnya kita mulai melatih model. Biasanya, dalam proses pembuatan model, kita akan memulai dengan model yang sederhana dan dengan rekayasa fitur yang minimum. Model sederhana yang pertama kali kita buat ini kita sebut sebagai model tolok-ukur atau *benchmark*. 

Setiap model berikutnya yang akan kita buat sebaiknya memiliki performa yang lebih baik secara signifikan dari model *benchmark*.

### Model *benchmark*

Kita akan menggunakan beberapa model sederhana sebagai model *benchmark*. Perbedaan perlakuan kita kepada model-model tersebut nanti hanya pada bagian bagaimana kita merekayasa fitur. Hal ini kita lakukan untuk melihat pengaruh rekayasa fitur terhadap performa model. Untuk itu, kita akan membuat beberapa fungsi bantuan.

Kebanyakan *library* memerlukan fitur dan label sebagai masukan terpisah. Oleh karena itu, kita perlu memisahkan fitur dan labelnya.

In [ ]:
def separate_data_and_label(df: DataFrame, label_col: str):
  ## <--- LENGKAPI --->
  ## Pertama, buanglah label_col dari list kolom-kolom di df. Simpan sisanya
  ## di dalam features.
  features = None

  ## Kemudian, ambil kolom-kolom di features dan simpan di data_df. 
  ## Terakhir, ambil kolom label_col dan simpan di label.
  data_df = None
  label = None
  
  return data_df, label

Kemudian, kita buat terlebih dahulu model `ColumnTransformer` untuk *one-hot encoding*.

In [ ]:
bm_column_tf = ColumnTransformer(
    [
     ('ohe-bm-1', OneHotEncoder(drop='first', sparse=False), categorical_columns)
    ],
    remainder='passthrough',
    verbose=True
)

Kita sudah dapat mulai melakukan pemrosesan. Secara umum, kita akan 
melakukan transformasi-transformasi yang membutuhkan informasi kolom `pandas` terlebih dahulu, diikuti dengan transformasi yang dilakukan oleh `scikit-learn`.

Pertama, mari kita proses data latih terlebih dahulu.

In [ ]:
bm_train_raw = train_df.drop(columns=dropped_cols)
bm_train_map = map_categorical_values(bm_train_raw, col_map_pairs)
bm_train_label_map = map_label(bm_train_map, label_col, '>50K', '<=50K')
bm_train_df, bm_train_label = separate_data_and_label(bm_train_label_map, label_col)

bm_column_tf.fit(bm_train_df)

bm_train_df_final = bm_column_tf.transform(bm_train_df)

Lalu kita proses data validasi. Perhatikan bahwa kita tidak perlu memanggil `fit` lagi untuk `bm_column_tf`.

In [ ]:
## <--- LENGKAPI --->
## Proseslah val_df dengan langkah-langkah seperti yang dilakukan terhadap
## train_df.  Anda tidak perlu memanggil bm_column_tf.fit() lagi.

bm_val_raw = None
bm_val_map = None
bm_val_label_map = None
bm_val_df, bm_val_label = None

bm_val_df_final = None

Karena `bm_train_df_final` dan `bm_val_df_final` bukan berupa `DataFrame` lagi, maka kita tidak bisa menggunakan `head()`. Kita cukup mengambil baris pertama saja dan menggunakan `print()` untuk mencetak.

In [ ]:
print(bm_train_df_final[0])

Saatnya memodelkan! Jalankan sel-sel berikut ini untuk memuat beberapa model beserta metrik performa model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

Kita akan mencoba memodelkan menggunakan model regresi logistik. Ingat bahwa kita memiliki `(bm_train_df, bm_train_label)`, fitur dan label data uji dan
`(bm_val_df, bm_val_label)`, fitur dan label data validasi.

Pertama, jalankan sel berikut untuk membuat objek `LogisticRegression`. Kita hanya akan memberikan argumen *constructor* berupa `max_iter`, yakni maksimum banyaknya iterasi. Anda dapat membuka [dokumentasi](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) `sklearn` untuk mengetahui hal-hal apa saja yang bisa diubah.

Pastikan bahwa konfigurasi model saat membuat objek sudah sesuai dengan yang Anda inginkan, misalnya:
- Jenis penalti regularisasi sudah sesuai (mis. `l2` atau `l1`).
- *Random state* sudah diisi apabila Anda ingin hasil Anda bisa direproduksi.
- Maksimum iterasi sudah sesuai.

In [ ]:
lr_model = LogisticRegression(max_iter=10000)

Berikutnya, untuk melatih model, kita dapat menggunakan metode `fit(X, y)`, dengan `X` adalah fitur-fitur di data latih dan `y` adalah label-labelnya.

In [ ]:
lr_model.fit(bm_train_df_final, bm_train_label)

Terakhir, untuk membuat prediksi terhadap data validasi, kita dapat menggunakan metode `predict`.

In [ ]:
lr_model_predictions = lr_model.predict(bm_val_df_final)

print(lr_model_predictions)

Karena kita memiliki label data uji, kita dapat melakukan evaluasi. Sebagai contoh, kita dapat menggunakan *method* `accuracy_score`.

In [ ]:
accuracy_score(bm_val_label, lr_model_predictions)

Daftar lengkap metrik performa yang dapat digunakan dapat Anda lihat di [dokumentasi](https://scikit-learn.org/stable/modules/classes.html?highlight=metrics#sklearn-metrics-metrics) modul `sklearn.metrics`

Di sel-sel berikut, kita akan membuat beberapa fungsi bantuan untuk melakukan evaluasi.

Pertama, `evaluate_prediction` akan menerima label dan hasil prediksi dari suatu model, kemudian menghitung dan mencetak hasil prediksi model tersebut.

In [ ]:
def evaluate_prediction(label: pd.Series, prediction: pd.Series):
  conf_matrix = confusion_matrix(label, prediction)
  acc = accuracy_score(label, prediction)
  precision, recall, f1, _ = precision_recall_fscore_support(label, prediction, average='binary')

  eval_result_template = f"""
  Confusion Matrix (True Class VS Predicted Class)
  {conf_matrix}

  Accuracy  : {acc}
  Precision : {precision}
  Recall    : {recall}
  F1-Score  : {f1}
  """
  
  print(eval_result_template)

Kemudian, `evaluate_model` akan menerima:
1. nama model,
1. objek model yang akan dilatih,
1. data latih tanpa label,
1. label data latih,
1. data uji tanpa label, dan
1. label data uji.

Fungsi `evaluate_model` ini kemudian akan melatih model menggunakan *method* `fit` dan melakukan prediksi dengan *method* `predict`. Setelah itu, hasil prediksinya akan dimasukkan ke dalam `evaluate_prediction` untuk dihitung performanya.

In [ ]:
def evaluate_model(model_name: str, model: object, train_df: np.ndarray, train_label: pd.Series, test_df: np.ndarray, test_label: pd.Series):
  ## <--- LENGKAPI --->
  ## Lakukan pelatihan terhadap objek model, kemudian lakukan prediksi terhadap
  ## test_df.
  None
  prediction = None
  
  print(f"""
  =================================================================
  {model_name} 
  <-------------------------------->
  """)
  evaluate_prediction(test_label, prediction)
  

Terakhir, kita dapat memperluas `evaluate_models` untuk memperumum `evaluate_model` agar bisa menerima banyak model.

In [ ]:
def evaluate_models(models: [object], train_df: np.ndarray, train_label: pd.Series, test_df: np.ndarray, test_label: pd.Series):
  for model_name, model in models:
    evaluate_model(model_name, model, train_df, train_label, test_df, test_label)

In [ ]:
models = [
    ('Logistic Regression', LogisticRegression()),
    ('K-Nearest Neighbours', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
]

evaluate_models(models, bm_train_df, bm_train_label, bm_val_df, bm_val_label)

### Performa model setelah *preprocessing* dan rekayasa fitur
Mari kita lihat apakah performa model kita bisa lebih baik setelah kita lakukan rekayasa fitur.

Umumnya, semua fungsi-fungsi yang berkaitan dengan *preprocessing* dan rekayasa fitur kita kumpulkan dalam satu fungsi besar seperti yang ada di sel berikut. Kumpulan fungsi-fungsi ini lazim disebut *pipeline* untuk data kita sebelum dimodelkan.

Untuk sesi ini, semua tahap *preprocessing* dan rekayasa fitur kita akan dikumpulkan pada fungsi `get_data_and_label()` dan `preprocess_data()`. 

Pertama, kita buat fungsi `get_data_and_label()` untuk:
1. membuang kolom,
1. menyederhanakan fitur kategorikal,
1. memetakan label ke 0/1,
1. melakukan binning,
1. memisahkan fitur dan label.

In [ ]:
def get_data_and_label(
    df: DataFrame,
    dropped_cols: [str],
    col_map_pairs: [str],
    col_bin_pairs: [(str, [float])],
    label_col: str,
    positive_label: str,
    negative_label: str):
  
  ## <--- LENGKAPI --->
  ## Lengkapi fungsi get_data_and_label(). Petunjuk fungsi apa yang harus Anda
  ## gunakan ada pada nama variabel. Perhatikan juga langkah-langkah yang 
  ## ditulis di atas.

  result_df = df.copy()
  result_df_dropped = None
  result_df_mapped_cat = None
  result_df_mapped_label = None
  result_df_binned = None
  data, label = None

  return data, label

Lalu kita manfaatkan fungsi tersebut untuk melakukan *preprocessing*.

In [ ]:
def preprocess_data(
    df: DataFrame,
    dropped_cols: [str],
    col_map_pairs: [(str, dict)],
    col_bin_pairs: [(str, [float])],
    label_col: str,
    positive_label: str,
    negative_label: str,
    column_transformer: ColumnTransformer):
  
  result_df = df.copy()

  ## <--- LENGKAPI --->
  ## Dapatkan dataframe yang telah di-preprocess dan dipisah fitur dan labelnya,
  ## kemudian gunakan column_transformer untuk mendapatkan data_final.

  data, label = None
  data_final = None
  
  return data_final, label
  

Pertama, kita gunakan `get_data_and_label` untuk memroses `train_df`, kemudian kita akan membuat `ColumnTransformer` untuk melakukan standardisasi dan *one-hot encoding* untuk data latih.

In [ ]:
train_raw, train_label = get_data_and_label(train_df, dropped_cols, col_map_pairs, col_bin_pairs, label_col, '>50K', '<=50K')

column_transformer = ColumnTransformer(
    [
      ('standard-scaling-1', StandardScaler(), standardized_cols),
      ('ohe-final-1', OneHotEncoder(drop='first', sparse=False), categorical_columns)
    ],
    remainder='passthrough',
    verbose=True
)

column_transformer.fit(train_raw)

train_df_final = column_transformer.transform(train_raw)

Kemudian, kita gunakan `preprocess_data()` dan `column_transformer` untuk memroses `val_df`.

In [ ]:
val_df_final, val_label = preprocess_data(val_df, dropped_cols, col_map_pairs, col_bin_pairs, label_col, '>50K', '<=50K', column_transformer)

Kita dapat kembali menggunakan `evaluate_models` untuk menguji model-model yang ingin kita buat terhadap data yang telah kita rekayasa.

In [ ]:
evaluate_models(models, prep_train_df_final, train_label, prep_val_df, val_label)

Jauh lebih baik! Sekarang kita akan masuk pada tahap selanjutnya, yakni *hyperparameter tuning*.

## Bagian 4 : *Hyperparameter tuning*

Dalam pembelajaran mesin, biasanya kita melakukan suatu proses optimisasi sehingga kita memperoleh parameter model yang memiliki galat minimum. Akan tetapi, untuk beberapa model, biasanya ada asumsi tertentu yang kita tetapkan sebelum memulai proses optimisasi. Pada model berbasis pohon, misalnya, kita menetapkan maksimal kedalaman pohon, atau pada model linear, kita menetapkan besarnya koefisien regularisasi. Asumsi-asumsi ini kita sebut sebagai *hyperparameter*.  

Tentu ada risiko bahwa kita tidak menemukan model yang terbaik karena kita salah mengambil asumsi tentang *hyperparameter* model kita. Untuk itu, kita melakukan proses *tuning*, yakni mencari *hyperparameter* yang akan memberikan hasil terbaik untuk model kita. Ada beberapa cara untuk melakukan ini.  Cara yang paling mudah adalah teknik *grid search*: mencoba beberapa kombinasi *hyperparameter* yang mungkin.

Berikut adalah contoh untuk regresi logistik:


In [ ]:
from sklearn.model_selection import GridSearchCV

model = LogisticRegression()
params = {'C': [0.01, 0.05, 0.1, 1.0, 10.0]}
lr_with_hyperparam_sets = GridSearchCV(model, param_grid=params, n_jobs=-1)
evaluate_model("Logistic regression hyperpameter tuning", lr_with_hyperparam_sets, train_df_final, train_label, val_df_final, val_label)

print('Best hyperparameters:\n', lr_with_hyperparam_sets.best_params_)

Dan berikut adalah contoh untuk *decision tree*.

In [ ]:
## <--- LENGKAPI --->
## Buat sebuah instance objek DecisionTreeClassifier tanpa argumen constructor.
dt_model = None

## Definisikan parameter grid search untuk GridSearchCV. Kita akan menggunakan:
## criterion: gini, entropy
## max_depth: 5, 10, 15, 20, 25, 30, 35, 40
## min_impurity_decrease: 0.0, 0.01, 0.05, 0.1, 0.25, 0.5

dt_params = None

## Buat objek GridSearchCV dengan model dt_model, param_grid dt_params, dan 
## n_jobs = -1.
dt_with_hyperparam_sets = None
evaluate_model("Decision tree hyperpameter tuning", dt_with_hyperparam_sets, train_df_final, train_label, val_df_final, val_label)

## Cetak hyperparameter terbaik dt_with_hyperparam_sets.
print('Best hyperparameters:\n', None)

## Bagian 5: Evaluasi model terhadap data uji

Setelah bersusah payah membuat model, tentu kita ingin tahu bagaimana kemampuan model kita membuat prediksi untuk data yang tidak pernah dilihat sebelumnya. Inilah saatnya kita menggunakan data uji.

In [ ]:
## <--- LENGKAPI --->
## Lakukan preprocessing terhadap test_df.

test_df_final, test_label = None

In [ ]:
print("Number of rows: {}".format(len(test_df_final)))

In [ ]:
prep_test_df, test_label = separate_data_and_label(prep_test_df)

Mari kita lihat performa model regresi logistik yang telah di-*tune* *hyperparameter*-nya.

In [ ]:
## <--- LENGKAPI --->
## Buatlah sebuah objek LogisticRegression yang baru menggunakan hyperparameter
## terbaik yang telah dihitung.
## Anda bisa memasukkan nilai-nilainya secara langsung, atau gunakan sintaks
## **dict (misalnya LogisticRegression(**params)) untuk secara otomatis 
## menggunakan sebuah dictionary sebagai ganti argumen ber-keyword.
##
## Setelah Anda membuat objek model, latih model tersebut dan lakukan prediksi
## terhadap prep_test_df.

model = None
None

test_prediction = None

In [ ]:
evaluate_prediction(test_label, test_prediction)

Untuk model-model yang lain, caranya pun serupa.

In [ ]:
models = [
    ('K-Nearest Neighbours', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
]

In [ ]:
evaluate_models(models, prep_train_df_final, train_label, prep_test_df, test_label)